# Hydrocarbon volumes calculator

The expression for _oil_ volumes in place:

$$ V = c \times GRV \times \phi \times NTG \times S_\mathrm{HC} \times \frac{1}{FVF} $$

where:

- c: conversion factor: `7758` from acre-ft to bbl (oil) or `43560` from acre-ft to ft3 (gas)
- V: hydrocarbon volumes
- GRV: Gross rock volume
- $\phi$: Porosity
- NTG: Net-to-gross
- $S_HC$: Hydrocarbon saturation (oil or gas)
- FVF: Formation volume factor


### Gross rock volume

Gross rock volume (GRV) can be expressed as:

$$GRV = A \times T \times G$$

where:

- A: Area
- T: Thickness
- G: Geometric correction factor

The [Geometric correction factor](https://subsurfwiki.org/wiki/Geometric_correction_factor) is applied to account for pinch out of the oil column at the edges of the prospect, it is a function of the thickness to closure height ratio as illustrated below:

<html>
    <img src="http://subsurfwiki.org/images/6/66/Geometric_correction_factor.png", width=600>
</html>

In [ ]:
# standard imports
import pandas as pd
pd.set_option("display.precision", 3)
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

In [ ]:
def calc_grv(thickness, height, area, top='slab', g=False):
    """Calculate GRV for given prospect
    Args:
        thickness [float]: average thickness of reservoir
        height [float]: height of hydrocarbon column
        area [float]: area of hydrocarbon prospect
        top: structure shape, one of `{'slab', 'round', 'flat'}`
        g [float]: geometric correction factor
    Returns:
        grv following `thickness * area * geometric_correction_factor`
    """
    if g:
        g = g
    else:
        ratio = thickness / height

        if top == 'round':
            g = -0.6 * ratio + 1
        elif top == 'flat':
            g = -0.3 * ratio + 1
        else:
            g = 1
    return thickness * area * g, g


def calc_hciip(GRV, phi=1, NTG=1, Sw=0, FVF=1, fluid='oil'):
    """Calculate Hydrocarbon initially in-place for a given prospect
    Args:
        GRV [float]: gross rock volume [acre-feet]
        phi [float]: porosity [fraction]
        NTG [float]: net-to-gross [fraction]
        Sw [float]: water saturation [fraction]
        FVF [float]: formation volume factor (Bo for oil or Bg for gas) [RES bbl/STB or RES ft3/SCF]
    """

    try:
        if fluid.lower() not in {'oil', 'gas'}:
            raise ValueError("`fluid` arg must be of `{'gas', 'oil'}`")
    except AttributeError:
        raise AttributeError("`fluid` arg must be of type `str`")

    if fluid == 'oil':
        constant = 7758 # conversion factor from acre-ft to bbl
    elif fluid == 'gas':
        constant = 43560 # conversion factor from acre-ft to ft3
   
    return (constant * GRV * phi * NTG * (1 - Sw)) / FVF

## Monte Carlo simulation of volumes

The two components of the Monte Carlo simulation are:
1. The equation to evaluate
2. The random variables for the input

Here the equation is `calc_hciip` and the variables are `{GRV, phi, NTG, Sw, FVF}`.

[Normal distribution applet](https://homepage.divms.uiowa.edu/~mbognar/applets/normal.html) and [Lognormal](https://homepage.divms.uiowa.edu/~mbognar/applets/lognormal.html) to eyeball parameters.

First, define the distributions paramters:

In [ ]:
df_raw = pd.read_clipboard(sep=',')
df_raw.index = df_raw.Reservoir
df_raw = df_raw.drop(columns='Reservoir')
df_raw

Then sample from a normal distribution (to begin with) and calculate volumes:

## Monte Carlo Experiment

In [ ]:
def plot_pdf(df, sand):
    """plot pdt of stoiip for given sand and save fig"""
    fig, ax = plt.subplots(figsize=(16,8))

    n, *_ = ax.hist(df.hciip, bins=30, alpha=0.4, color='green')
    n_len = (len(str(int(n.max()))))
    y_max = round(n.max(), -(n_len-1))

    title = f'Probability density function of STOIIP for {sand}'
    ax.set_title(title, fontsize=26)

    minvol, maxvol = df.hciip.describe().loc[['min', 'max']]

    for name, prob in {'P90': p90, 'P50': p50, 'P10': p10}.items():
        formatted_prob = round(prob, -3)/1e6
        prob_approx = f'\n{formatted_prob:.1f}\nmmbbl'
        plt.axvline(prob, c='b', linewidth=1.5)
        ax.text(prob, y_max - y_max*.2, name + prob_approx, fontdict={'fontsize': 22})

    ax.set_xlim(minvol, maxvol)
    ax.tick_params(axis='both', which='major', labelsize=20)
    plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
    plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

    ax.set_xlabel('STOIIP [bbl]', fontsize=20)
    ax.set_ylabel('Density of probability [1 / bbl]', fontsize=20)

    plt.savefig(f'./stoiip_pdf_{sand}', dpi=400)
    
    return None

In [ ]:
for sand in df_raw.index:
    grv_avg, grv_std = df_raw.loc[sand, 'grv_avg'], df_raw.loc[sand, 'grv_std']
    phi_avg, phi_std = df_raw.loc[sand, 'phi_avg'], df_raw.loc[sand, 'phi_std']
    ntg_avg, ntg_std = df_raw.loc[sand, 'ntg_avg'], df_raw.loc[sand, 'ntg_std']
    sw_avg, sw_std = df_raw.loc[sand, 'sw_avg'], df_raw.loc[sand, 'sw_std']
    fvf_avg, fvf_std = df_raw.loc[sand, 'fvf_avg'], df_raw.loc[sand, 'fvf_std']
    # sample count for MC experiment
    num_samples = 5000
    # Choose random inputs for each variable
    ### NOTE THAT OTHER DISTIBUTIONS CAN BE SUBSTITUTED, SEE FOR EXAMPLE: https://docs.scipy.org/doc/scipy/reference/stats.html#continuous-distributions
    grv = np.random.normal(grv_avg, grv_std, num_samples)
    phi = np.random.normal(phi_avg, phi_std, num_samples)
    ntg = np.random.normal(ntg_avg, ntg_std, num_samples)
    sw = np.random.normal(sw_avg, sw_std, num_samples)
    fvf = np.random.normal(fvf_avg, fvf_std, num_samples)
    # set negative values to 0
    for arr in [grv, phi, ntg, sw, fvf]:
        arr[arr < 0] = 0
    # set values over 1 to 1
    for arr in [phi, ntg, sw]:
        arr[arr > 1] = 1
    # Build the dataframe based on the inputs and num_samples
    df = pd.DataFrame(index=range(num_samples), data={'grv': grv,'phi': phi,'ntg': ntg,'sw': sw,'fvf': fvf,})
    # calculate HCIIPs based on inputs
    df['hciip'] = calc_hciip(df.grv, df.phi, df.ntg, df.sw, df.fvf)
    # reverse p90, p50, p10 to match oil-field convention
    p90, p50, p10 = df.describe(percentiles=[.1,.5,.9]).loc[['10%', '50%', '90%'], 'hciip']
    df_raw.loc[sand, 'P90'] = p90 / 1e6
    df_raw.loc[sand, 'P50'] = p50 / 1e6
    df_raw.loc[sand, 'P10'] = p10 / 1e6
    plot_pdf(df, sand)

In [ ]:
field = input('what is the field name?')

In [ ]:
col_units = ['acre-feet',
             'acre-feet',
             '%',
             '%',
             '%',
             '%',
             '%',
             '%',
             'bbl/STB',
             'bbl/STB',
             'MMSTB',
             'MMSTB',
             'MMSTB',
            ]
cols_multiindex = list(zip(df_raw.columns, col_units))
df_raw.columns = pd.MultiIndex.from_tuples(cols_multiindex)

In [ ]:
df_raw.loc['Totals'] = df_raw.loc[:,['grv_avg', 'P90', 'P50', 'P10']].sum(axis=0)
df_raw

In [ ]:
df_raw.to_excel(f'./{field}_quicklook_volumetrics.xlsx')